<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myProspects_Upside_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

gen_output = 0

Run date time (IST): 2024-05-01 13:15:46


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta
!pip install pyxirr
#!pip install talib-binary

In [3]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

from google.colab import data_table

In [4]:
# set start and end date
start_date = (datetime.date.today() + datetime.timedelta(days=-10*365)).strftime('%Y-%m-%d')
start1_date = (datetime.date.today() + datetime.timedelta(days=-1*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [5]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

def read_file(path):
 df = pd.read_csv(path)
 return df

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/'

mypfs = pd.read_csv(path_mypf + 'myPortfolioStocks.csv')
mypps = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

df_mypf = pd.merge(mypfs, mypps, how='right', on="Symbol")


def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'ADANITRANS', 'SREERAYALK', 'SCHAEFFLER ', 'EQUITAS']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

df_mypf = df_mypf[df_mypf['InPortfolio'].isnull()]
#cols = ['Symbol', 'Criteria', 'Strategy', 'LatestQtr', 'Target','Breakout','StarStock', 'Conviction', 'Category', 'Stock', 'MedianPE']
#df_mypf = df_mypf[cols]
my_prospects = get_stock_ids(df_mypf)
df_stocks = df_mypf[df_mypf['Symbol'].isin(my_prospects)]

In [6]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_features(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND', 'BANDHANBNK', 'ABBOTINDIA', 'JCHAC',
              'NESTLEIND', 'PGHH', 'EQUITASBNK', 'SBICARD', 'ASTRAZEN', 'ICICIGI', 'RELAXO',
              'HDFCAMC', 'STARHEALTH', 'BAYERCROP', 'BIRLAMONEY', 'TATAELXSI', 'APEX', 'SPORTKING',
            'CASTROLIND', 'HONAUT']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:

    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
    current_roce = float(test_list[test_list.index('ROCE')+1:test_list.index('ROCE')+2][0])
    current_roe = float(test_list[test_list.index('ROE')+1:test_list.index('ROE')+2][0])
  except ValueError as ve:
    current_pe = 1000
    current_roce = 1000
    current_roe = 1000

  return current_pe, current_roce, current_roe

In [7]:
common_cols = ['Symbol', 'Target', 'Breakout', 'MedianPE', 'NSECap', 'Criteria', 'Strategy', 'TFrame', 'LatestQtr', 'StarStock', 'BizFinTech', 'Conviction', 'Category']
df_stocks = df_stocks[common_cols]

In [8]:

# function to get prospects technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='10y', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start1_date)]
    stock_df['Min'] = round(min(stock_df['Close']),2)
    #stock_df['Gained%'] = round((stock_df['Close'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    #stock_df['Close'] = round(stock_df['Close'],0)
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'
                  ], axis=1, inplace=True)

    return stock_df

# function to get common features for prospects
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in stock_ids:
      #print(stock_id)
      #current_pe = get_current_pe(stock_id)
      current_pe, current_roce, current_roe = get_current_features(stock_id)
      tmp = stock_prec_dev(stock_id)
      tmp['Prev_Close'] = tmp['Close'].shift(1)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      tmp['Criteria'] = df_mypf[(df_mypf['Symbol']) == stock_id]['Criteria'].values[0]
      tmp['Strategy'] = df_mypf[(df_mypf['Symbol']) == stock_id]['Strategy'].values[0]
      tmp['Target'] = df_mypf[(df_mypf['Symbol']) == stock_id]['Target'].values[0]
      tmp['Breakout'] = df_mypf[(df_mypf['Symbol']) == stock_id]['Breakout'].values[0]
      tmp['LatestQtr'] = df_mypf[(df_mypf['Symbol']) == stock_id]['LatestQtr'].values[0]
      tmp['StarStock'] = df_mypf[(df_mypf['Symbol']) == stock_id]['StarStock'].values[0]
      tmp['Conviction'] = df_mypf[(df_mypf['Symbol']) == stock_id]['Conviction'].values[0]
      tmp['Category'] = df_mypf[(df_mypf['Symbol']) == stock_id]['Category'].values[0]
      tmp['Med_PE'] = df_mypf[(df_mypf['Symbol']) == stock_id]['MedianPE'].values[0]
      tmp['Curr_PE'] = current_pe
      tmp['Dev%_PE'] = round((tmp['Curr_PE'] - tmp['Med_PE'])*100/tmp['Med_PE'],2)
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
      #df_prec_dev = df_prec_dev.append(tmp)

  return df_prec_dev

stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

In [9]:
df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)

In [10]:
# prospects stocks

def get_prospects_features(df_common_features):

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = np.where(df_common_features['Breakout'].isnull(), df_common_features['Min'], df_common_features["Breakout"])

  #tmp_df = df_common_features[df_common_features['Stock'].isin(df_mypf[df_mypf['InPortfolio'] != 'NA'].Stock.values)]
  tmp_df = df_common_features
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))

  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)

  return tmp_df

df_prospects_features = get_prospects_features(df_common_features)

qualified stocks: 152
with latest results: 121
still star stocks: 70


In [11]:
cols = ['Symbol', 'Close', 'NTT',  'Dev%_200', 'Dev%_PE', 'RSI_14', 'NTT%', 'LTT%', 'Gained%',  'Criteria', 'Strategy','LatestQtr', 'StarStock', 'Conviction', 'Category']
df_prospects_features = df_prospects_features[cols]

if gen_output == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  df_prospects_features.to_csv('/content/drive/My Drive/data/stocks/myStocks-Portfolio-Analysis.csv', index=False)

In [12]:
# avsp
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['AVSP'])].sort_values(by = 'LTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,NTT,Dev%_200,Dev%_PE,RSI_14,NTT%,LTT%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category
142,UTIAMC,950.45,1207.00,14.51,-21.78,60.0,26.99,18.48,30.20,AVSP,NTT,1,0,M,FINANCE
120,SHALBY,277.30,327.00,6.25,37.31,58.0,17.92,16.50,105.74,AVSP,NTT,1,0,H,HEALTHCARE
80,JSWHL,6766.40,7316.45,25.84,42.55,47.0,8.13,10.65,63.38,AVSP,BTT,1,0,H,FINANCE


In [13]:
# av40
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['AV40'])].sort_values(by = 'LTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,NTT,Dev%_200,Dev%_PE,RSI_14,NTT%,LTT%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category
59,HDFCLIFE,583.65,754.39,-7.50,-10.74,35.0,29.25,53.69,8.70,AV40,BTT,1,0,H,INSURANCE
106,PFIZER,4257.20,5855.00,3.41,247.95,52.0,37.53,40.97,13.75,AV40,NTT,0,0,H,PHARMA
50,GILLETTE,6667.90,7564.13,8.05,-12.75,59.0,13.44,34.68,57.87,AV40,BTT,0,1,H,FMCG
57,HCLTECH,1366.60,1686.40,-0.03,19.80,25.0,23.40,23.40,32.85,AV40,ATH,1,1,H,IT
51,GLAXO,2086.75,2466.80,18.90,-1.61,60.0,18.21,18.21,73.43,AV40,ATH,1,0,H,PHARMA
6,AKZOINDIA,2449.85,3300.00,-3.89,-22.86,49.0,34.70,17.54,7.43,AV40,NTT,1,1,H,NaN
107,PGHH,16170.30,18360.87,-2.50,-11.78,51.0,13.55,13.55,21.63,AV40,ATH,0,0,H,FMCG
18,BAJAJHLDNG,8116.15,9149.60,5.68,-3.88,47.0,12.73,12.73,31.07,AV40,ATH,1,0,H,FINANCE
1,ABBOTINDIA,26446.05,27500.00,7.40,-1.60,49.0,3.99,11.69,27.33,AV40,NTT,0,1,H,PHARMA
75,ITC,435.65,485.03,-0.54,22.02,58.0,11.33,11.33,8.01,AV40,ATH,1,1,H,FMCG


In [14]:
# av40n
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['AV40N'])].sort_values(by = 'LTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,NTT,Dev%_200,Dev%_PE,RSI_14,NTT%,LTT%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category
77,JCHAC,1265.55,3219.25,10.72,1823.08,66.0,154.38,154.38,34.51,AV40N,ATH,0,0,M,AC
145,VINATIORGA,1636.60,1882.00,-5.67,13.11,57.0,14.99,42.59,11.38,AV40N,NTT,0,0,M,CHEMICALS
8,ANGELONE,2786.60,3791.17,9.02,10.95,46.0,36.05,36.05,137.26,AV40N,ATH,1,1,M,FINANCE
146,VIPIND,549.60,740.05,-7.63,54.19,59.0,34.65,34.65,21.35,AV40N,ATH,1,0,M,MISC
29,CERA,7190.90,9546.30,-10.66,0.50,49.0,32.76,32.76,14.01,AV40N,ATH,1,1,M,CERAMICS
12,ASTRAZEN,5335.30,6927.05,8.91,-17.80,51.0,29.83,29.83,69.40,AV40N,ATH,0,1,M,PHARMA
0,3MINDIA,30468.10,37309.15,-1.34,-13.48,54.0,22.45,22.45,36.56,AV40N,ATH,1,1,M,MISC
44,EQUITASBNK,97.05,114.55,0.05,-20.69,47.0,18.03,18.03,38.86,AV40N,ATH,0,1,M,BANKS
26,CAPLIPOINT,1356.45,1597.70,12.94,34.09,56.0,17.79,17.79,101.33,AV40N,ATH,1,1,M,PHARMA
102,OFSS,7598.65,8949.65,42.67,78.92,42.0,17.78,17.78,122.66,AV40N,ATH,1,1,M,IT


In [15]:
# av200, avr and aar, exp50
df_tmp = df_prospects_features[~df_prospects_features['Criteria'].isin(['AV40','AV40N'])].sort_values(by = 'LTT%', ascending=False)
df_tmp = df_tmp[df_tmp['Criteria'].notnull()]
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,NTT,Dev%_200,Dev%_PE,RSI_14,NTT%,LTT%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category
9,APEX,216.65,314.00,-2.41,286.88,55.0,44.93,295.14,-3.71,AAR,NTT,0,0,L,MISC
24,CAMPUS,247.35,631.85,-9.73,-26.35,57.0,155.45,155.45,15.31,AAR,ATH,0,1,L,FOOTWEAR
35,DCBBANK,139.70,168.00,10.87,10769.57,68.0,20.26,68.42,32.91,AAR,NTT,1,1,L,BANKS
104,PARAGMILK,211.95,273.00,1.41,34.62,46.0,28.80,64.76,99.95,AAR,NTT,1,0,L,FMCG
141,UPL,507.15,NaN,-9.28,NaN,62.0,NaN,61.34,12.23,AAR,NTT,1,0,L,NaN
109,PNB,141.05,90.00,53.32,22.89,68.0,-36.19,50.97,194.90,AVR,NTT,1,1,M,BANKS
41,DREAMFOLKS,544.70,815.64,-0.06,-6.72,63.0,49.74,49.74,27.19,EXP50,ATH,0,0,L,MISC
34,CUB,160.70,200.00,15.61,5997.56,69.0,24.46,49.01,20.83,AAR,NTT,0,1,L,BANKS
71,INFIBEAM,34.00,19.45,41.67,NaN,43.0,-42.79,45.44,147.99,AAR,NTT,1,1,L,IT
99,NIACL,247.15,278.00,31.46,87.18,59.0,12.48,44.72,147.65,AVR,NTT,1,0,M,INSURANCE


In [16]:
# all prospects
df_prospects_features = df_prospects_features.sort_values(by = 'LTT%', ascending=False)
data_table.DataTable(df_prospects_features, include_index=False)

,Symbol,Close,NTT,Dev%_200,Dev%_PE,RSI_14,NTT%,LTT%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category
150,YESBANK,26.15,388.67,24.52,NaN,59.0,1386.31,1386.31,68.71,NaN,NaN,1,0,L,NaN
151,ZEEL,146.95,588.15,-34.69,163.41,48.0,300.24,300.24,6.06,NaN,NaN,1,0,L,ENTERTAINMENT
9,APEX,216.65,314.00,-2.41,286.88,55.0,44.93,295.14,-3.71,AAR,NTT,0,0,L,MISC
24,CAMPUS,247.35,631.85,-9.73,-26.35,57.0,155.45,155.45,15.31,AAR,ATH,0,1,L,FOOTWEAR
77,JCHAC,1265.55,3219.25,10.72,1823.08,66.0,154.38,154.38,34.51,AV40N,ATH,0,0,M,AC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,HDFCAMC,3894.65,3903.85,26.61,7.83,65.0,0.24,-7.57,129.10,AV40,BTT,1,1,H,FINANCE
119,SBIN,826.25,826.50,28.90,-18.94,70.0,0.03,-11.04,51.05,AVR,BTT,1,1,M,BANKS
116,SAIL,164.20,168.55,52.04,154.43,68.0,2.65,-12.30,102.74,AVR,BTT,1,0,M,STEEL
79,JIOFIN,377.00,387.95,41.20,NaN,59.0,2.90,-13.00,82.48,AVR,BTT,0,0,M,FINANCE
